Finetune a Pretrained Hugginface model

In [ ]:
!pip install transformers
!pip install transformers[torch]

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
  

In [ ]:
!pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 21.2 MB/s eta 0:00:00


In [ ]:
#Import Libraries
import os

#Data manipulation
import pandas as pd
import numpy as np
import datasets
from sklearn.model_selection import train_test_split
import collections

import re

#finetuning
from transformers import AutoTokenizer
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import Trainer

#Evaluation
from datasets import load_metric, load_dataset
from sklearn.metrics import f1_score
from huggingface_hub import notebook_login

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#login to huggingface
notebook_login()

Activate weight and Biases

In [ ]:
os.environ["WANDB_DISABLED"] = "false"

Load The Datasets

In [ ]:
#load data
import pandas as pd

train_df=pd.read_csv('/content/drive/MyDrive/Nors/Noise/Fine-grained Noise/train_dev.csv',sep = ',')

#check first 5

train_df


,text,label
0,Bag efter dem fulgte et halvthundrede eller tr...,M
1,Toget fra Korsør havde lige standset sin Fart....,H
2,alle Kirkens Aabninger. Indenfor lød Kirkesang...,M
3,"Langt borte støjer Selskabet, Violinens Toner ...",H
4,Begunstiget af Regnen og det tætte Mørke flygt...,T
...,...,...
1134,"9 Byens Gader bølgede Menneskemængden, 3S som ...",H
1135,Idet han kastede Adelhejd over imod Væggen var...,T
1136,Kirkeklokkerne ringe i Prag. Gaderne er strøed...,T
1137,Kirkeklokken ringede til Eftermiddags-Gudstjen...,T


In [ ]:
# Create a dictionary to map the values
value_map = {
    'H': '0',
    'T': '1',
    'N': '2',
    'M': '3'
}

# Apply the mapping to the column
train_df['label'] = train_df['label'].map(value_map)

In [ ]:
train_df

,text,label
0,Bag efter dem fulgte et halvthundrede eller tr...,3
1,Toget fra Korsør havde lige standset sin Fart....,0
2,alle Kirkens Aabninger. Indenfor lød Kirkesang...,3
3,"Langt borte støjer Selskabet, Violinens Toner ...",0
4,Begunstiget af Regnen og det tætte Mørke flygt...,1
...,...,...
1134,"9 Byens Gader bølgede Menneskemængden, 3S som ...",0
1135,Idet han kastede Adelhejd over imod Væggen var...,1
1136,Kirkeklokkerne ringe i Prag. Gaderne er strøed...,1
1137,Kirkeklokken ringede til Eftermiddags-Gudstjen...,1


In [ ]:
#view info
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1139 entries, 0 to 1138
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1139 non-null   object
 1   label   1139 non-null   object
dtypes: object(2)
memory usage: 17.9+ KB


# Remove the rows with null values in the label and agreement columns of the train_df DataFrame

In [ ]:
# Remove rows with null values
train_df.dropna(subset=['label', 'text'], inplace=True)

In [ ]:
# View info once again
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1139 entries, 0 to 1138
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1139 non-null   object
 1   label   1139 non-null   object
dtypes: object(2)
memory usage: 17.9+ KB


**Danish Text Preprocessing**

In [ ]:
#danish text preproccessing
import re
import nltk
import string

nltk.download('punkt')
nltk.download("stopwords")

def preprocess_danish(text):
    # Lowercase the text
    text = text.lower()
    # Remove punctuation
    text = text.translate(text.maketrans("", "", string.punctuation))
    # text = re.sub(r'[^a-zA-ZæøåÆØÅ]', ' ', text)
    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove stopwords (optional)
    #nltk.download('stopwords')
    from nltk.corpus import stopwords
    #stop_words = set(stopwords.words("danish"))
    #tokens = [word for word in tokens if word not in stop_words]

    return (" ".join(tokens))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


**Apply Text Preprocessing**

In [ ]:
# Apply the preprocessing function to the 'text_column' using apply
print(train_df['text'][:2])
train_df['text'] = train_df['text'].apply(preprocess_danish)
print()
print(train_df['text'][:2])

0    Bag efter dem fulgte et halvthundrede eller tr...
1    Toget fra Korsør havde lige standset sin Fart....
Name: text, dtype: object

0    bag efter dem fulgte et halvthundrede eller tr...
1    toget fra korsør havde lige standset sin fart ...
Name: text, dtype: object


In [ ]:
print(train_df['text'][:20])

0     bag efter dem fulgte et halvthundrede eller tr...
1     toget fra korsør havde lige standset sin fart ...
2     alle kirkens aabninger indenfor lød kirkesange...
3     langt borte støjer selskabet violinens toner l...
4     begunstiget af regnen og det tætte mørke flygt...
5     neen dryssede fin og tæt ned over torvet folk ...
6     med ravnsorte lokker der duftede af pomade og ...
7     overalt straalede en uhyre mængde lys fra de g...
8     efter komediens ophør begyndte dandsen et mægt...
9     det er ikke naturskønheder der henriver øjet m...
10    en strøm af glade mennesker gled uafbrudt forb...
11    laugebrogadc er en af de mest afsides og mest ...
12    men mørket kom og klokken ringede ud de høje p...
13    en halv time senere rullede en droske bort fra...
14    han hastede op gennem byen det var lige i mørk...
15    brutto vilde gøre endnu flere indvendinger men...
16    kun kunde spille danse direktøren var fortvivl...
17    det elektriske ringeapparat skingrede genn

## Revert to original Dataset

In [ ]:
train_df.drop(train_df.columns.difference(['text', 'label']), axis=1, inplace=True)

AS Noticed in EDA we will drop the rows with missing values

In [ ]:
#Drop null rows
train_df.dropna(axis=0,inplace=True)

#check for the presence of null values again
train_df.isna().sum()


text     0
label    0
dtype: int64

Split the Train the dataset into training(for learning ) and evaluation(For computing metrics to capture overfitting) sets

In [ ]:
# train , eval = train_test_split(train_df,train_size=0.8,stratify=train_df['label'])
train = train_df.iloc[:960]
eval = train_df.iloc[961:]


check train data

In [ ]:
train.head()

,text,label
0,bag efter dem fulgte et halvthundrede eller tr...,3
1,toget fra korsør havde lige standset sin fart ...,0
2,alle kirkens aabninger indenfor lød kirkesange...,3
3,langt borte støjer selskabet violinens toner l...,0
4,begunstiget af regnen og det tætte mørke flygt...,1


Check eval

In [ ]:
eval.head()

,text,label
961,ved lyden af hans stemme foer hun forskrækket op,2
962,3der var sang og skraal latter og graad som bl...,0
963,musiken bifaldet bruser,0
964,musiken blandede sig i al den pludder og staah...,0
965,ja „ rhythmos “ var der i spillet det var sikk...,0


Check the shapes of both dataset

In [ ]:
print(f'Shape of training set: {train.shape}\nShape of evaluation set: {eval.shape}')

Shape of training set: (960, 2)
Shape of evaluation set: (178, 2)


Save the splitted data to be loaded as huggingface dataset

In [ ]:
# Save splitted subsets
train.to_csv("/content/drive/MyDrive/Nors/Noise/Fine-grained Noise/Data/train_subset.csv", index=False)
eval.to_csv("/content/drive/MyDrive/Nors/Noise/Fine-grained Noise/Data/eval_subset.csv", index=False)

In [ ]:
datasets = load_dataset('csv',
                        data_files={'train':'/content/drive/MyDrive/Nors/Noise/Fine-grained Noise/Data/train_subset.csv',
                        'eval':'/content/drive/MyDrive/Nors/Noise/Fine-grained Noise/Data/eval_subset.csv'}, encoding = "utf-8")

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

In [ ]:
print(datasets['train'])

Dataset({
    features: ['text', 'label'],
    num_rows: 960
})


## Creating a Tokenizer instance

In [ ]:
#Create a tokenizer instance
tokenizer=AutoTokenizer.from_pretrained('MiMe-MeMo/MeMo-BERT-03')

tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

## Functions to tokenize text and transform labels

In [ ]:
#function to tokenize text
def tokenize_data(data):
    return tokenizer(data['text'],padding='max_length')


#function to transform labels
def transfom_label(data):

    #extract label
    label=data['label']
    num=0
    #create contitions
    if label==0:
        num=0
    elif label==1:
        num=1
    elif label==2:
        num=2
    elif label==3:
        num=3
    return {'labels':num}

## Tokenizer text and trasform original sentiment labels

In [ ]:
#The following columns will be removed after tokenization
remove_columns = ['label', 'text']

#Tokenize the text data
datasets=datasets.map(tokenize_data,batched=True)

#transform the labels
datasets=datasets.map(transfom_label,remove_columns=remove_columns)

Map:   0%|          | 0/960 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.


Map:   0%|          | 0/178 [00:00<?, ? examples/s]

Map:   0%|          | 0/960 [00:00<?, ? examples/s]

Map:   0%|          | 0/178 [00:00<?, ? examples/s]

## Extract the train and eval datasets from datasets

In [ ]:
#extract train datasets
train_dataset=datasets['train'].shuffle(seed=0)
#extract eval datasets
eval_dataset=datasets['eval'].shuffle(seed=0)

## Define Metrics Function

In [ ]:
# Define the function to compute F1-score
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='weighted')
    return {"f1-score": f1}

## Three models shall be trained in turns as follows:
1. 'bert-base-cased' and be named as: 'tweet_sentiments_analysis_bert'
2. 'distilbert-base-cased' and be named as: 'tweet_sentiments_analysis_distilbert'
3. 'roberta-base' and be named as: 'tweet_sentiments_analysis_roberta'

## Set the training argumets

In summary, the TrainingArguments object is used to define various training configurations and hyperparameters, such as the number of training epochs, evaluation and saving strategies, and whether to push the model to the Hugging Face Model Hub after training. This object is then passed to the Trainer class for model training using the specified settings.

In [ ]:
#set the training arguments
trainargs=TrainingArguments('/content/drive/MyDrive/Nors/Noise/Fine-grained Noise/Models/MEMOBERT3-Base_FGN',
                            num_train_epochs=20,
                            evaluation_strategy="epoch",
                            save_strategy='epoch',
                            load_best_model_at_end=True,
                            metric_for_best_model = 'f1-score',
                            push_to_hub=True)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Create an instance of the Model

In [ ]:
# Create an instance of the BERT model
model_name = 'MiMe-MeMo/MeMo-BERT-03'
num_labels = 4
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, trust_remote_code=True)

configuration_norbert.py:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ltg/norbert3-base:
- configuration_norbert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_norbert.py:   0%|          | 0.00/27.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ltg/norbert3-base:
- modeling_norbert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/522M [00:00<?, ?B/s]

## Create trainer instance

In [ ]:
# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=trainargs,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    )

## Start training

In [ ]:
trainer.train()

In [ ]:
# Evaluate the model
results = trainer.evaluate()
print("BERT model:")
print("Evaluation results:", results)

BERT model:
Evaluation results: {'eval_loss': 1.4464290142059326, 'eval_f1-score': 0.8360565174143689, 'eval_runtime': 1.6406, 'eval_samples_per_second': 108.5, 'eval_steps_per_second': 14.02, 'epoch': 20.0}


## Push model to huggingface

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1717754985.a5a0751fcd91.635.5:   0%|          | 0.00/411 [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1717753651.a5a0751fcd91.635.4:   0%|          | 0.00/13.4k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.47G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/yemen2016/NorBERT3-Large_FGN/commit/5d28b2f229ac2cf2a3fb4fb535f0b5dd84eefb40', commit_message='End of training', commit_description='', oid='5d28b2f229ac2cf2a3fb4fb535f0b5dd84eefb40', pr_url=None, pr_revision=None, pr_num=None)

## Evaluate the model on testing set

In [ ]:
from transformers import pipeline
clf = pipeline("text-classification", '-------/nb-bert-base_FGN')

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
new_text = preprocess_danish('» For nu skal vi jo til at gøre hinandens bekendtskab ,og det gør man allerbedst med et glas i hånden !')
answer = clf(new_text)
print(answer)

[{'label': 'LABEL_0', 'score': 0.9955132603645325}]


In [ ]:
#load data
import pandas as pd

test_df=pd.read_csv('/content/drive/MyDrive/Nors/Noise/Fine-grained Noise/test.csv',sep = ',')


test_df

,text,label
0,"Grupper, smøgende dercsMiber, drikkende og Pra...",H
1,"Der blev holdt taler og udbragt skåler, og lid...",H
2,en summende hvisken fra de kurtiserende og kok...,H
3,Det var Byfogdens Stemme med sin ejendommelige...,H
4,"Sangforeninger, dels improviserede, dels af me...",H
...,...,...
100,Kirkeklokkerne ringede. Drøn af Kanonskud genl...,T
101,"Han hørte Vingeslagene og saa de mørke, udspæn...",T
102,Klokkerne løde til Arbeide paa Pladserne og Sk...,T
103,Han stod nok saa ugenert og passiarede ind gen...,T


In [ ]:
# Create a dictionary to map the values
value_map = {
    'H': '0',
    'T': '1',
    'N': '2',
    'M': '3'
}

# Apply the mapping to the column
test_df['label'] = test_df['label'].map(value_map)

In [ ]:
test_df

,text,label
0,"Grupper, smøgende dercsMiber, drikkende og Pra...",0
1,"Der blev holdt taler og udbragt skåler, og lid...",0
2,en summende hvisken fra de kurtiserende og kok...,0
3,Det var Byfogdens Stemme med sin ejendommelige...,0
4,"Sangforeninger, dels improviserede, dels af me...",0
...,...,...
100,Kirkeklokkerne ringede. Drøn af Kanonskud genl...,1
101,"Han hørte Vingeslagene og saa de mørke, udspæn...",1
102,Klokkerne løde til Arbeide paa Pladserne og Sk...,1
103,Han stod nok saa ugenert og passiarede ind gen...,1


In [ ]:
# Using readlines()
texts = test_df['text'].tolist()
labels = test_df['label'].tolist()
count = 0
acutal_list = []
predicted_list= []
# Strips the newline character
for text,label in zip(texts, labels):
    count += 1
    # print("Line{}: {}".format(count, line.strip()))
    # print(label)
    sentence = text
    actual_sentiment = label
    acutal_list.append(actual_sentiment)
    new_text = preprocess_danish(sentence)
    answer = clf(new_text)
    predicted_sentiment = answer[0]['label'][6:]
    predicted_list.append(predicted_sentiment)
    print(str(predicted_sentiment) + "\t" + str(actual_sentiment))

0	0
0	0
0	0
3	0
3	0
3	0
3	0
0	0
0	0
0	0
3	0
0	0
0	0
3	0
1	0
0	0
0	0
0	0
3	0
0	0
3	0
0	0
0	0
0	0
0	0
3	0
0	0
1	0
0	0
0	0
1	0
0	0
0	0
0	0
0	0
3	0
3	3
3	3
3	3
3	3
3	3
3	3
3	3
3	3
3	3
3	3
3	3
3	3
3	3
3	3
3	3
3	3
3	3
3	3
3	3
0	3
3	3
0	2
1	1
1	1
1	1
1	1
3	1
0	1
1	1
1	1
1	1
1	1
1	1
1	1
1	1
0	1
1	1
1	1
1	1
1	1
1	1
1	1
1	1
1	1
1	1
1	1
1	1
1	1
1	1
1	1
1	1
1	1
1	1
1	1
1	1
1	1
1	1
1	1
0	1
1	1
1	1
1	1
1	1
1	1
1	1
3	1
1	1
1	1
1	1


In [ ]:
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score
import numpy as np

# Assuming acutal_list and predicted_list are already defined
# Calculate the F1-score for each class
f1_scores = f1_score(acutal_list, predicted_list, average=None)

# Calculate the weighted F1-score
weighted_f1_score = np.average(f1_scores, weights=np.bincount(acutal_list))

# Calculate the confusion matrix
conf_matrix = confusion_matrix(acutal_list, predicted_list)

# Calculate precision and recall for each class
precision = precision_score(acutal_list, predicted_list, average=None)
recall = recall_score(acutal_list, predicted_list, average=None)

print("F1-scores for each class:", f1_scores)
print("Weighted F1-score:", weighted_f1_score)
print("Confusion Matrix:\n", conf_matrix)
print("Precision for each class:", precision)
print("Recall for each class:", recall)


F1-scores for each class: [0.71875    0.91304348 0.         0.75471698]
Weighted F1-score: 0.8060676198288995
Confusion Matrix:
 [[23  3  0 10]
 [ 3 42  0  2]
 [ 1  0  0  0]
 [ 1  0  0 20]]
Precision for each class: [0.82142857 0.93333333 0.         0.625     ]
Recall for each class: [0.63888889 0.89361702 0.         0.95238095]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
